## Colormaps - SDK examples

This notebook shows how to:
- List objects in an Evo workspace.
- Create a new colormap and associate it with an object attribute.
- Download a colormap and inspect it's properties.
- All code samples use the **Evo Python SDK**.

### Requirements

You must have a Seequent account with the Evo entitlement to use this notebook.

The following parameters must be provided:

- The client ID of your Evo application.
- The callback/redirect URL of your Evo application.

To obtain these app credentials, refer to the [Apps and tokens guide](https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens) in the Seequent Developer Portal.

In [ ]:
from evo.colormaps import ColormapAPIClient
from evo.notebooks import ServiceManagerWidget
from evo.objects import ObjectAPIClient

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    redirect_url=redirect_url,
    client_id=client_id,
).login()

### Use the Evo Python SDK to create an object client and a data client

In [ ]:
# The object client will manage your auth token and Geoscience Object API requests.
object_client = ObjectAPIClient(manager.get_environment(), manager.get_connector())

# The colormap client will manage your colormap API requests.
colormap_client = ColormapAPIClient(manager.get_environment(), manager.get_connector())

# The data client will manage saving your data as Parquet and publishing your data to Evo storage.
data_client = object_client.get_data_client(manager.cache)

### Display all objects in the workspace

In [ ]:
from collections import defaultdict

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout

all_objects = await object_client.list_all_objects()

if len(all_objects) == 0:
    print("No objects found.")
else:
    # Group objects by schema classification/sub-classification for a two-level picker
    def object_type(obj):
        sub_cls = getattr(obj.schema_id, "sub_classification", None)
        if isinstance(sub_cls, (list, tuple)) and sub_cls:
            return " / ".join(sub_cls)
        if isinstance(sub_cls, str) and sub_cls:
            return sub_cls
        cls = getattr(obj.schema_id, "classification", None)
        return cls or "Unknown"

    grouped = defaultdict(list)
    for obj in all_objects:
        grouped[object_type(obj)].append(obj)

    for objs in grouped.values():
        objs.sort(key=lambda o: o.name)

    type_options = sorted(grouped.keys())

    def build_object_options(selected_type):
        return [(o.name.removesuffix(".json"), str(o.id)) for o in grouped[selected_type]]

    # Dynamic widths based on content lengths
    def width_for_labels(labels, min_px=220, max_px=720, char_px=8):
        longest = max((len(lbl) for lbl in labels), default=0)
        return f"{min(max(min_px, longest * char_px + 40), max_px)}px"

    type_width = width_for_labels(type_options)
    object_width = width_for_labels([name for name, _ in build_object_options(type_options[0])])
    label_width = "110px"
    row_gap = "6px"

    type_dropdown = widgets.Dropdown(
        options=type_options,
        description="",
        disabled=False,
        layout=widgets.Layout(width=type_width),
    )

    objects_dropdown = widgets.Dropdown(
        options=build_object_options(type_options[0]),
        description="",
        disabled=False,
        layout=widgets.Layout(width=object_width),
    )

    def on_type_change(change):
        if change.get("name") == "value" and change.get("new") in grouped:
            new_options = build_object_options(change["new"])
            objects_dropdown.options = new_options
            objects_dropdown.layout.width = width_for_labels([name for name, _ in new_options])

    type_dropdown.observe(on_type_change, names="value")

    label_layout = Layout(width=label_width, display="flex", justify_content="flex-start", align_items="center")
    row_layout = Layout(align_items="center", gap=row_gap)
    type_row = widgets.HBox([widgets.Label("Object type", layout=label_layout), type_dropdown], layout=row_layout)
    objects_row = widgets.HBox([widgets.Label("Object", layout=label_layout), objects_dropdown], layout=row_layout)

    print(f"Found {len(all_objects)} object(s) across {len(type_options)} type(s). Select a type, then an object:")
    display(widgets.VBox([type_row, objects_row]))

### Select an attribute from the chosen object

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Get the selected object ID
selected_object_id = objects_dropdown.value

# Download the object to extract its attributes
object_data = await object_client.download_object_by_id(object_id=selected_object_id)
object_dict = object_data.as_dict()


def extract_attributes(data, attributes=None):
    """Recursively extract all attributes with 'key' and 'name' from the object data."""
    if attributes is None:
        attributes = []

    if isinstance(data, dict):
        # Check if this dict has both 'key' and 'name' properties
        if "key" in data and "name" in data:
            attributes.append({"key": str(data.get("key")), "name": data.get("name")})
        # Recursively search all values in the dict
        for value in data.values():
            extract_attributes(value, attributes)
    elif isinstance(data, list):
        # Recursively search all items in the list
        for item in data:
            extract_attributes(item, attributes)

    return attributes


# Extract all attributes from the object
all_attributes = extract_attributes(object_dict)

if len(all_attributes) == 0:
    print("No attributes found in this object.")
else:
    # Create dropdown options with (display_name, attribute_key) and sort alphabetically
    attribute_options = sorted([(attr["name"], attr["key"]) for attr in all_attributes], key=lambda x: x[0])

    attribute_dropdown = widgets.Dropdown(
        options=attribute_options,
        description="Attribute:",
        disabled=False,
        style={"description_width": "initial"},
    )

    print(f"Found {len(all_attributes)} attribute(s) in the selected object. Choose one:")
    display(attribute_dropdown)

### Create and publish a category colormap

In [ ]:
from evo.colormaps import CategoryColormap
from evo.colormaps.data import Association

# Get the selected attribute ID from the dropdown
selected_attribute_name = attribute_dropdown.label
selected_attribute_id = attribute_dropdown.value

# Define the category colormap with sample categories and colors
categories = ["Rock Type A", "Rock Type B", "Rock Type C", "Rock Type D"]
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]  # RGB tuples

# Create a category colormap
category_colormap = CategoryColormap(
    map=categories,
    colors=colors,
)

# First, create the colormap in Evo
colormap_metadata = await colormap_client.create_colormap(
    colormap=category_colormap,
    name=f"Category colormap for {selected_object_id}",
)

print("Colormap created successfully!")
print(f"Colormap ID: {colormap_metadata.id}")

# Then, associate the colormap with the object attribute
association = Association(
    attribute_id=selected_attribute_id,
    colormap_id=colormap_metadata.id,
)

association_metadata = await colormap_client.create_association(
    object_id=selected_object_id,
    association=association,
)

print(f"Colormap associated with object attribute: {selected_attribute_name}")
print(f"Association ID: {association_metadata.id}")

### Fetch colormap data

Find all colormap associations for the chosen object.

In [ ]:
import json
from datetime import datetime
from uuid import UUID

from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer


def serialize_for_display(obj):
    """Convert an object's attributes to a JSON-serializable dictionary."""
    if isinstance(obj, UUID):
        return str(obj)
    elif isinstance(obj, datetime):
        return obj.isoformat()
    elif isinstance(obj, list):
        return [serialize_for_display(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: serialize_for_display(value) for key, value in obj.items()}
    elif hasattr(obj, "__dict__"):
        return {key: serialize_for_display(value) for key, value in vars(obj).items()}
    else:
        return obj


object_id = objects_dropdown.value
associations = await colormap_client.get_association_collection(object_id)

if associations is None or len(associations) == 0:
    print("\nNo colormap associations found for this object.")
else:
    print("\nColormap associations for this object:\n")
    for association in associations:
        serialized = serialize_for_display(association)
        print(highlight(json.dumps(serialized, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

Iterate through all colormaps to learn what the colormap type is, eg. category, continuous, or discrete, then cross-reference the colormap `attribute ID` with the attributes in the chosen geoscience object.

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

# Prepare a combined list of collections from planned and interim data.
downloaded_object = await object_client.download_object_by_id(object_id=object_id)

downloaded_dict = downloaded_object.as_dict()


def find_attribute_by_key(data, target_key):
    """Recursively search through data structure to find an item with matching key."""
    if isinstance(data, dict):
        # Check if this dict has a 'key' property matching our target
        if "key" in data and str(data.get("key")) == str(target_key):
            return data.get("name")
        # Recursively search all values in the dict
        for value in data.values():
            result = find_attribute_by_key(value, target_key)
            if result is not None:
                return result
    elif isinstance(data, list):
        # Recursively search all items in the list
        for item in data:
            result = find_attribute_by_key(item, target_key)
            if result is not None:
                return result
    return None


for association in associations:
    colormap_id = association.colormap_id

    colormap_metadata = await colormap_client.get_colormap_by_id(colormap_id)

    print(f"\n{'=' * 80}")
    print(f"Colormap ID: {colormap_id}")
    print(f"{'=' * 80}\n")

    # Cross-reference the attribute ID with the downloaded object to find the attribute name.
    attribute_id = association.attribute_id
    attribute_name = find_attribute_by_key(downloaded_dict, attribute_id)

    colormap = colormap_metadata.colormap
    colormap_type = colormap.__class__.__name__

    print(f"{colormap_type} for attribute: {attribute_name or 'Unknown'}\n")

    if colormap_type == "CategoryColormap":
        map = colormap.map
        colors = colormap.colors

        # Normalize colors from 0-255 range to 0-1 range for matplotlib
        normalized_colors = [tuple(c / 255.0 for c in color) for color in colors]

        # Create visualization
        fig, ax = plt.subplots(figsize=(10, len(colors) * 0.5))
        ax.set_xlim(0, 1)
        ax.set_ylim(0, len(colors))
        ax.axis("off")

        for i, (label, color) in enumerate(zip(map, normalized_colors)):
            y_pos = len(colors) - i - 1
            # Draw color rectangle
            rect = mpatches.Rectangle((0, y_pos), 0.15, 0.8, facecolor=color, edgecolor="black", linewidth=0.5)
            ax.add_patch(rect)
            # Add label text
            ax.text(0.18, y_pos + 0.4, f"{label}", va="center", fontsize=10)
            print(f"{label}: {colors[i]}")

        plt.title(f"Category colormap for attribute: {attribute_name or 'Unknown'}", fontsize=12, pad=10, loc="left")
        plt.tight_layout()
        plt.show()

    elif colormap_type == "DiscreteColormap":
        colors = colormap.colors
        end_points = colormap.end_points
        end_inclusive = colormap.end_inclusive

        print("Colors:")
        for color in colors:
            print(f"{color}")

        print("End points:", end_points)
        print("End inclusive:", end_inclusive)

    elif colormap_type == "ContinuousColormap":
        attribute_controls = colormap.attribute_controls
        gradient_controls = colormap.gradient_controls
        colors = colormap.colors

        print(f"Attribute controls: {attribute_controls}")
        print(f"Gradient controls: {gradient_controls}")
        print(f"Number of colors: {len(colors)}")

    else:
        print("Unknown colormap type")